## Imports

In [4]:
import numpy as np
import imageio
import cv2
import json

## Seeding

Assumes pre-defined mask listed in mask_notebook. Can be updated directly on the file 

## Main Program

Loads image

In [3]:
# Load image
image_path = "dataset_examples/um_000052.png"
image = imageio.imread(image_path)

In [4]:
cv2.imshow("Original image", image)
cv2.waitKey()
cv2.destroyAllWindows()

## Image Pre-processing

Applies a series of transformations to input image in order to 

In [10]:
# Select bottom half of the image and convert it to HSV
image_height = image.shape[0]
cropped_image = image[image_height // 2: image_height, :, :]
hsv_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)

In [11]:
# Apply mask listed in mask_notebook.json
masked_images = []
with open("mask_notebook.json") as masks_file:
    masks = json.load(masks_file)
    for mask in masks:
        min_range = np.array(mask["min"])
        max_range = np.array(mask["max"])
        masked_images.append(cv2.inRange(image, min_range, max_range))

    masked_image = np.sum(masked_images, axis=0) / len(masked_images)
    masked_image[masked_image != 0] = 255
    masked_image = np.uint8(masked_image)

In [12]:
blurred_image = cv2.GaussianBlur(masked_image, (3, 3), 0)

In [13]:
kernel = np.ones([5, 5], np.uint8)
closed_masked_image = cv2.erode(cv2.dilate(blurred_image, kernel), kernel)

In [14]:
canny_image = cv2.Canny(closed_masked_image, 50, 150)

In [15]:
edged_image = cv2.erode(cv2.dilate(blurred_image, kernel), kernel)

Show all processed images

In [17]:
cv2.imshow("Cropped", cropped_image)
cv2.imshow("HSV", hsv_image)
cv2.imshow("Mask", masked_image)
cv2.imshow("Blurred", blurred_image)
cv2.imshow("Closed Blurred", closed_masked_image)
cv2.imshow("Canny", canny_image)
cv2.imshow("Edged", edged_image)
cv2.waitKey()
cv2.destroyAllWindows()

## Contour processing

Find contours in image

In [ ]:
contours, _ = cv2.findContours(edged_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cont_image = np.zeros([edged_image.shape[0], edged_image.shape[1], 3])
approximated_contours = []

for contour in contours:
    if cv2.contourArea(contour) < 10: continue

    epsilon = 0.01 * cv2.arcLength(contour, True)
    approximated_contour = cv2.approxPolyDP(contour, epsilon, True)
    convex_contour = cv2.convexHull(approximated_contour)
    approximated_contours.append(convex_contour)
    cv2.drawContours(cont_image, [convex_contour], -1, (0, 255, 0), 1)

Show contours

In [ ]:
cv2.imshow("Contours", cont_image)
cv2.waitKey()
cv2.destroyAllWindows()

Show contours

In [ ]:
cv2.imshow("Contours", cont_image)
cv2.waitKey()
cv2.destroyAllWindows()